In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import env
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy import stats

from math import sqrt

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# imports for modeling:
# import Logistic regression
from sklearn.linear_model import LogisticRegression
# import K Nearest neighbors:
from sklearn.neighbors import KNeighborsClassifier
# import Decision Trees:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
# import Random Forest:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix

# interpreting our models:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#feature selection
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector


<h1>Feature Engineering Exercises<h1/>

<h3>Load the 'tips' dataset.

<h3/>

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


<h3>a. Create a column named price_per_person. This should be the total bill divided by the party size.
<h3/>

In [4]:
df['price_per_person'] = df['total_bill' ]/ df['size']
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


<h3>b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
<h3/>

In [5]:
# size, total_bill

<h3>c. Use Select K Best to select the top 2 features for predicting tip amount. What are they?
<h3/>

In [6]:
df.head(2)

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667


In [64]:
def clean_tips(df):
    dummy_df = pd.get_dummies(df[['sex', 'day', 'time', 'smoker']], drop_first=False)
    cleaned_df = pd.concat([df, dummy_df], axis = 1)
    return cleaned_df

def machine_tips(cleaned_df):
    machine_df = cleaned_df.drop(columns = ['sex', 'day', 'time', 'smoker'])
    
    
    import sklearn.preprocessing
    # 1. create the object
    scaler = sklearn.preprocessing.MinMaxScaler()
    
    # 2. fit the object (learn the min and max value)
    scaler.fit(machine_df[['total_bill', 'price_per_person', 'size']])
    
    # 3. use the object (use the min, max to do the transformation)
    
    scaled_stuff = scaler.transform(machine_df[['total_bill', 'price_per_person', 'size']])
    
    machine_df[['total_bill_scaled', 'price_per_person_scaled', 'size_scaled']] = scaled_stuff
    
    machine_df = machine_df.drop(columns = ['total_bill', 'price_per_person', 'size'])
    return machine_df

In [51]:
cleaned_df = clean_tips(df)
cleaned_df.head(2)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,smoker_Yes,smoker_No
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000,0,1,0,0,0,1,0,1,0,1
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667,1,0,0,0,0,1,0,1,0,1


In [65]:
machine_df = machine_tips(cleaned_df)
machine_df

,tip,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,smoker_Yes,smoker_No,total_bill_scaled,price_per_person_scaled,size_scaled
0,1.01,0,1,0,0,0,1,0,1,0,1,0.291579,0.322989,0.2
1,1.66,1,0,0,0,0,1,0,1,0,1,0.152283,0.032854,0.4
2,3.50,1,0,0,0,0,1,0,1,0,1,0.375786,0.237261,0.4
3,3.31,1,0,0,0,0,1,0,1,0,1,0.431713,0.515230,0.2
4,3.61,0,1,0,0,0,1,0,1,0,1,0.450775,0.188075,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,5.92,1,0,0,0,1,0,0,1,0,1,0.543779,0.390900,0.4
240,2.00,0,1,0,0,1,0,0,1,1,0,0.505027,0.615805,0.2
241,2.00,1,0,0,0,1,0,0,1,1,0,0.410557,0.486207,0.2
242,1.75,1,0,0,0,1,0,0,1,0,1,0.308965,0.346839,0.2


In [66]:
def split_data(machine_df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    
    # Create train_validate and test datasets
    train_validate, test = train_test_split(machine_df, test_size=0.2, random_state=123)
    
    # Create train and validate datsets
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=123)

    # Take a look at your split datasets

    print(f"""
    train -> {train.shape}
    validate -> {validate.shape}
    test -> {test.shape}""")
    
    return train, validate, test

In [67]:
train, validate, test = split_data(machine_df)
train.head()


    train -> (136, 14)
    validate -> (59, 14)
    test -> (49, 14)


,tip,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,smoker_Yes,smoker_No,total_bill_scaled,price_per_person_scaled,size_scaled
18,3.50,0,1,0,0,0,1,0,1,0,1,0.291160,0.159866,0.4
172,5.15,1,0,0,0,0,1,0,1,1,0,0.087558,0.043103,0.2
118,1.80,0,1,1,0,0,0,1,0,0,1,0.196062,0.191954,0.2
28,4.30,1,0,0,0,1,0,0,1,0,1,0.390239,0.458333,0.2
237,1.17,1,0,0,0,1,0,0,1,1,0,0.623377,0.778161,0.2


In [68]:
X_train=train.drop(columns='tip')
y_train=train.tip

X_validate=validate.drop(columns='tip')
y_validate=validate.tip

X_test=test.drop(columns='tip')
y_test=test.tip

In [69]:
# MAKE the thing
kbest = SelectKBest(f_regression, k=2)

# FIT the thing
kbest.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x15b788a40>)

In [70]:
# statistical f-value / feature's scores:
kbest.scores_

array([1.15479204e+00, 1.15479204e+00, 1.07637964e+00, 6.92417587e-01,
       3.18953072e-03, 2.40040378e+00, 1.79864746e+00, 1.79864746e+00,
       3.44908618e-01, 3.44908618e-01, 1.15984909e+02, 1.07777918e+01,
       6.12590886e+01])

In [71]:
# p value: 
kbest.pvalues_

array([2.84479443e-01, 2.84479443e-01, 3.01377404e-01, 4.06823770e-01,
       9.55046793e-01, 1.23662519e-01, 1.82144884e-01, 1.82144884e-01,
       5.57997755e-01, 5.57997755e-01, 7.18647033e-20, 1.31032690e-03,
       1.34164190e-12])

In [72]:
kbest.feature_names_in_

array(['sex_Male', 'sex_Female', 'day_Thur', 'day_Fri', 'day_Sat',
       'day_Sun', 'time_Lunch', 'time_Dinner', 'smoker_Yes', 'smoker_No',
       'total_bill_scaled', 'price_per_person_scaled', 'size_scaled'],
      dtype=object)

In [73]:
kbest_results = pd.DataFrame(
                dict(p=kbest.pvalues_, f=kbest.scores_),
                                        index = X_train.columns)

In [74]:
kbest_results

,p,f
sex_Male,2.844794e-01,1.154792
sex_Female,2.844794e-01,1.154792
day_Thur,3.013774e-01,1.076380
day_Fri,4.068238e-01,0.692418
day_Sat,9.550468e-01,0.003190
day_Sun,1.236625e-01,2.400404
time_Lunch,1.821449e-01,1.798647
time_Dinner,1.821449e-01,1.798647
smoker_Yes,5.579978e-01,0.344909
smoker_No,5.579978e-01,0.344909


In [75]:
# get-support() will output a boolean mask to tell me which features were selected
kbest.get_support()

array([False, False, False, False, False, False, False, False, False,
       False,  True, False,  True])

In [76]:
# we can apply this mask to the columns in our original dataframe
X_train.columns[kbest.get_support()]

Index(['total_bill_scaled', 'size_scaled'], dtype='object')

In [77]:
# Let's turn the KB-selected features into a DF
X_train_KBtransformed = pd.DataFrame(
    kbest.transform(X_train),
    columns=X_train.columns[kbest.get_support()],
    index=X_train.index
)

In [78]:
# The results of KB transformed and ready for modeling if these were the only two features we wanted to look at

X_train_KBtransformed.head()

,total_bill_scaled,size_scaled
18,0.291160,0.4
172,0.087558,0.2
118,0.196062,0.2
28,0.390239,0.2
237,0.623377,0.2


<h3>d. Use Recursive Feature Elimination to select the top 2 features for tip amount. What are they?
<h3/>

In [79]:
from sklearn.linear_model import LinearRegression

# make a model object to use in RFE process.
# The model is here to give us metrics on feature importance and model score
# allowing us to recursively reduce the number of features to reach our desired space

model = LinearRegression()

In [80]:
# MAKE the thing
rfe = RFE(model, n_features_to_select=2)

# FIT the thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [81]:
# Get feature ranking
# Selected features are assigned a rank 1

rfe.ranking_

array([ 6,  7,  5,  4,  1,  1,  2,  3,  9,  8, 10, 11, 12])

In [82]:
# Dataframe of rankings

pd.DataFrame(
{
    'rfe_ranking':rfe.ranking_
}, index = X_train.columns)

,rfe_ranking
sex_Male,6
sex_Female,7
day_Thur,5
day_Fri,4
day_Sat,1
day_Sun,1
time_Lunch,2
time_Dinner,3
smoker_Yes,9
smoker_No,8


In [83]:
X_train_RFEtransformed = pd.DataFrame(
                            rfe.transform(X_train),
                            index = X_train.index,
                            columns = X_train.columns[rfe.support_])

In [84]:
X_train_RFEtransformed.head()

,day_Sat,day_Sun
18,0.0,1.0
172,0.0,1.0
118,0.0,0.0
28,1.0,0.0
237,1.0,0.0


<h3>e. Why do you think Select K Best and Recursive Feature Elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?
<h3/>

<h4>**Answer: I think I got different answers for the top features because Select K Best is the fastest, but not the most accurate option, while RFE took its time and weighed day_Thurs as a better feature to predict tip amount**<h4/>

<h3>Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

<h3/>

In [85]:
def select_kbest(X_train, y_train, k):
    # MAKE the thing
    kbest = SelectKBest(f_regression, k=k)

    # FIT the thing
    kbest.fit(X_train, y_train)
    return X_train.columns[kbest.get_support()]

In [86]:
select_kbest(X_train, y_train, 2)

Index(['total_bill_scaled', 'size_scaled'], dtype='object')

<h3>Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top n features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

<h3/>

In [87]:
def rfe(X_train, y_train, k):
    model = LinearRegression()
    
    # MAKE the thing
    rfe = RFE(model, n_features_to_select=k)
    
    # FIT the thing
    rfe.fit(X_train, y_train)
    
    X_train_RFEtransformed = pd.DataFrame(
                            rfe.transform(X_train),
                            index = X_train.index,
                            columns = X_train.columns[rfe.support_])
    
    return X_train_RFEtransformed.columns

In [88]:
rfe(X_train, y_train, 2)

Index(['day_Sat', 'day_Sun'], dtype='object')

<h3>Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both Select K Best and Recursive Feature Elimination (use the functions you just built to help you out).

<h3/>

In [89]:
swiss = pd.read_csv('swiss.csv')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [34]:
swiss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [35]:
def split_data(swiss):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    
    # Create train_validate and test datasets
    train_validate, test = train_test_split(swiss, test_size=0.2, random_state=123)
    
    # Create train and validate datsets
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=123)

    # Take a look at your split datasets

    print(f"""
    train -> {train.shape}
    validate -> {validate.shape}
    test -> {test.shape}""")
    
    return train, validate, test

In [36]:
train, validate, test = split_data(swiss)
train.head()


    train -> (25, 6)
    validate -> (12, 6)
    test -> (10, 6)


,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,60.5,60.8,16,10,7.72,16.3
Lavaux,65.1,73.0,19,9,2.84,20.0
Nyone,56.6,50.9,22,12,15.14,16.7
Conthey,75.5,85.9,3,2,99.71,15.1
Yverdon,65.4,49.5,15,8,6.10,22.5


In [37]:
X_train=train.drop(columns='Fertility')
y_train=train.Fertility

X_validate=validate.drop(columns='Fertility')
y_validate=validate.Fertility

X_test=test.drop(columns='Fertility')
y_test=test.Fertility

In [38]:
#Select K Best - Top 3 results
select_kbest(X_train, y_train, 3)

Index(['Examination', 'Catholic', 'Infant.Mortality'], dtype='object')

In [39]:
#Recursive Feature Elimination - Top  3 results
rfe(X_train, y_train, 3)

Index(['Agriculture', 'Examination', 'Infant.Mortality'], dtype='object')